In [170]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [171]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [172]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [173]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [174]:
df = pd.merge(ratings, movies, on = 'movieId', how = 'left')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [175]:
# Create a user-item matrix
movie_ratings = df.pivot(index='userId', columns='movieId', values='rating').fillna(0)
movie_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [176]:
user_ratings = movie_ratings.T
user_ratings.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [177]:
# Standardize the data
scaler = StandardScaler()
ratings_scaled = scaler.fit_transform(user_ratings)
ratings_scaled

array([[ 5.74894663, -0.05362189, -0.04837714, ...,  2.39705526,
        14.70276093,  3.48053624],
       [-0.15372814, -0.05362189, -0.04837714, ...,  1.86011842,
        -0.06123831, -0.38147487],
       [ 5.74894663, -0.05362189, -0.04837714, ...,  1.86011842,
        -0.06123831, -0.38147487],
       ...,
       [-0.15372814, -0.05362189, -0.04837714, ..., -0.28762896,
        -0.06123831, -0.38147487],
       [-0.15372814, -0.05362189, -0.04837714, ..., -0.28762896,
        -0.06123831, -0.38147487],
       [-0.15372814, -0.05362189, -0.04837714, ..., -0.28762896,
        -0.06123831, -0.38147487]])

In [178]:
# Compute similarity matrix
movie_similarity = cosine_similarity(ratings_scaled)

In [179]:
# Create a DataFrame for item similarity matrix
similarity = pd.DataFrame(movie_similarity, index=user_ratings.index, columns=user_ratings.index)
similarity

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.241590,0.221517,-0.047995,0.263772,0.264502,0.278710,0.005502,0.294387,0.201349,...,-0.160958,-0.183205,-0.143203,-0.143203,-0.160958,-0.143203,-0.160958,-0.160958,-0.160958,-0.155363
2,0.241590,1.000000,0.162869,-0.004338,0.232735,0.160211,0.147334,0.134859,0.001428,0.306088,...,-0.135396,-0.153995,-0.120550,-0.120550,-0.135396,-0.120550,-0.135396,-0.135396,-0.135396,-0.130807
3,0.221517,0.162869,1.000000,-0.003218,0.392015,0.364073,0.339700,0.300072,0.373545,0.117894,...,-0.069055,-0.078381,-0.061609,-0.061609,-0.069055,-0.061609,-0.069055,-0.069055,-0.069055,-0.066878
4,-0.047995,-0.004338,-0.003218,1.000000,0.021547,-0.024038,0.185922,0.056166,-0.009409,-0.001281,...,0.030962,0.036173,0.026821,0.026821,0.030962,0.026821,0.030962,0.030962,0.030962,0.028942
5,0.263772,0.232735,0.392015,0.021547,1.000000,0.379655,0.384691,0.196556,0.369247,0.135424,...,-0.052798,-0.059887,-0.047137,-0.047137,-0.052798,-0.047137,-0.052798,-0.052798,-0.052798,-0.051174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,-0.143203,-0.120550,-0.061609,0.026821,-0.047137,-0.108296,-0.055753,0.014047,-0.010314,-0.109909,...,0.998950,0.994538,1.000000,1.000000,0.998950,1.000000,0.998950,0.998950,0.998950,0.102039
193583,-0.160958,-0.135396,-0.069055,0.030962,-0.052798,-0.121641,-0.062490,0.016556,-0.011240,-0.123474,...,1.000000,0.998276,0.998950,0.998950,1.000000,0.998950,1.000000,1.000000,1.000000,0.116520
193585,-0.160958,-0.135396,-0.069055,0.030962,-0.052798,-0.121641,-0.062490,0.016556,-0.011240,-0.123474,...,1.000000,0.998276,0.998950,0.998950,1.000000,0.998950,1.000000,1.000000,1.000000,0.116520


In [180]:
def recommend_items(movie_id, num_recommendations=5):
    # Get similar items
    similar_movie = similarity[movie_id].sort_values(ascending=False)
    
    # Exclude the input item from recommendations
    similar_movie = similar_movie.drop(movie_id)
    
    # Get the top similar items
    top_similar_movie = similar_movie.head(num_recommendations)
    
    return top_similar_movie

In [181]:
# Example usage
movie_id = int(input("Enter the item ID: "))
recommendations = recommend_items(movie_id)
print(f"If you watched {df['title'][movie_id-1]}, then you may also like: ")
for movie_id, similarity in recommendations.items():
    title = df[df['movieId'] == movie_id]['title'].values[0] if movie_id in df['movieId'].values else 'Unknown'
    print(f"{title}")

If you watched Toy Story (1995), then you may also like: 
Independence Day (a.k.a. ID4) (1996)
Mission: Impossible (1996)
Twister (1996)
Willy Wonka & the Chocolate Factory (1971)
Nutty Professor, The (1996)
